# Backtest Example

In this notebook i will demonstrate a simple backtesting strategy over the QQQ etf

Buying criteria:
- 10 ema and 21 ema sloping upward
- 10 ema above 21 ema

Selling criteria:
- 10 ema and 21 ema sloping down
- 10 ema below 21 ema

In [20]:
from reports.tradeLog import TradeLog
from datetime import datetime
import backtrader as bt
from backtrader.analyzers import AnnualReturn, TradeAnalyzer
import yfinance as yf

from reports.tradeLog import TradeLog
from reports.strategySummery import build_summary_report
import exports.exporter as exporter
class SmaCross(bt.Strategy):
    params = dict(
        period_fast=10,
        period_slow=21
    )

    def __init__(self):
        self.sma10 = bt.ind.SMA(period=self.p.period_fast)
        self.sma21 = bt.ind.SMA(period=self.p.period_slow)

    def next(self):
        sma10_above_sma21 = self.sma10[0] > self.sma21[0]
        sma10_sloping_up = self.sma10[0] > self.sma10[-1] if self.sma10[-1] else False
        sma21_sloping_up = self.sma21[0] > self.sma21[-1] if self.sma21[-1] else False

        if not self.position:
            if sma10_above_sma21 and sma10_sloping_up and sma21_sloping_up:
                self.buy()

        elif not sma10_above_sma21 and not sma10_sloping_up and not sma21_sloping_up:
            self.close()


import exports.exporter as ex

print("Exporter file:", ex.__file__)
print("Dir exporter:", dir(ex))


cerebro = bt.Cerebro()
cerebro.addsizer(bt.sizers.AllInSizer)
cerebro.broker.setcash(100000)

qqq = yf.download("QQQ",  start=datetime(2017, 1, 1),end=datetime(2024, 12, 31))
qqq.columns = ["Open", "High", "Low", "Close", "Volume"] # Compatability with backtrader pandasData format
qqq_datafeed = bt.feeds.PandasData(dataname=qqq)
cerebro.adddata(qqq_datafeed, name="QQQ")

cerebro.addstrategy(SmaCross)
cerebro.addanalyzer(TradeLog, _name='trade_log')

cerebro.addanalyzer(AnnualReturn)
cerebro.addanalyzer(TradeAnalyzer)

sma_cross_strategy = cerebro.run()[0]

trade_table = sma_cross_strategy.analyzers.trade_log.get_analysis()
strategy_name = sma_cross_strategy.__class__.__name__
report = build_summary_report(sma_cross_strategy)
log_path = exporter.export_trade_log_csv(trade_table, strategy_name)
summary_csv_path = exporter.export_summary_report_csv(report, strategy_name)

#filename = f"trade_log_{strategy_name}.csv"
#keys = trade_table[0].keys()

cerebro.plot()
sma_cross_strategy.analyzers[0].get_analysis()
# with open(filename, "w", newline="") as f:
#     writer = csv.DictWriter(f, fieldnames=keys)
#     writer.writeheader()
#     writer.writerows(trade_table)

/var/folders/g8/lzy7w86j4w393km21jt_s8400000gn/T/ipykernel_9980/1330449873.py:43: FutureWarning: YF.download() has changed argument auto_adjust default to True
  qqq = yf.download("QQQ",  start=datetime(2017, 1, 1),end=datetime(2024, 12, 31))
[*********************100%***********************]  1 of 1 completed

Exporter file: /Users/noamfishbain/PycharmProjects/code-bulls/exports/exporter.py
Dir exporter: ['Any', 'Dict', 'List', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'csv', 'export_trade_log_csv']


AttributeError: module 'exports.exporter' has no attribute 'export_summary_report_csv'